#### **Chargement des Claims dans Bronze**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, when

In [2]:
# Create Spark session
spark = SparkSession.builder \
                    .appName("Healthcare Cpt_codes Ingestion") \
                    .getOrCreate()

In [3]:
BUCKET_NAME = "healthcare-bucket-lte"
CPT_BUCKET_PATH = f"gs://{BUCKET_NAME}/landing/cpt_codes/*.csv"
BQ_TABLE = "gcp-demo-lte.bronze_dataset.cpt_codes"
TEMP_GCS_BUCKET = f"{BUCKET_NAME}/temp/"



In [4]:
# read from claims source
cpt_codes_df = spark.read.csv(CPT_BUCKET_PATH, header=True) \
                .dropDuplicates()

for old_name in cpt_codes_df.columns:
    new_name = old_name.replace(" ", "_").lower()
    cpt_codes_df = cpt_codes_df.withColumnRenamed(old_name, new_name)

In [5]:
# write to bigquery
(cpt_codes_df.write
            .format("bigquery")
            .option("table", BQ_TABLE)
            .option("temporaryGcsBucket", TEMP_GCS_BUCKET)
            .mode("overwrite")
            .option("overwriteSchema", True)
            .save())